# Chapter 2.6. Intro to TensorFlow and Keras

Original code in Nikolenko book was made in **TensorFlow 1.x (TF1)**, but here I will convert examples to **TensorFlow 2.0 (TF2)**

In [1]:
import numpy as np
import tensorflow as tf

from models_classes import RegModel
from losses import mse_loss

## 1. Simple Regression using TensorFlow 2

$$y = k_{true} * x + b_{true} + \epsilon$$

$$\epsilon - normal noise (N(0, 2))$$

Let's try to recover coefs `k_true`, `b_true`

In [2]:
# set constants
n_samples = 1000
batch_size = 100
num_steps = 2000
display_step = 100

k_true = 2
b_true = 1

# generate data
X_data = np.random.uniform(1,10,(n_samples, 1))
y_data = k_true * X_data + b_true + np.random.normal(0, 2, (n_samples, 1))

# initialize coefs
k = tf.Variable(tf.random.normal((1,1), dtype='float64'), name='slope')
b = tf.Variable(tf.zeros((1,), dtype='float64'), name='bias')

# define train process
def train(model, X, y, optimizer):
    """
    One step of train process
    
    params:
        model: model object ot train
        X: data
        y: target
        optimizer: tf optimizer
    return:
        loss: loss value
    """
    with tf.GradientTape() as t:
        y_pred = model(X)
        loss = mse_loss(y, y_pred)
        
    grads = t.gradient(loss, [model.k, model.b])
    optimizer.apply_gradients(zip(grads,[model.k, model.b]))
    
    return loss

In [3]:
optimizer = tf.optimizers.SGD(learning_rate=1e-4)
model = RegModel(k, b)

# running optimization
for i in range(num_steps):
    
    # select data batch
    indices = np.random.choice(n_samples, batch_size)
    X_batch, y_batch = X_data[indices], y_data[indices]
    
    loss_val = train(model, X_batch, y_batch, optimizer)
    
    # output info
    if (i+1) % display_step == 0:
        print(f'Epoch {i+1}: loss={loss_val}, k={model.k.numpy()}, b={model.b.numpy()}')

Epoch 100: loss=414.14657788073606, k=[[2.07025512]], b=[0.56551404]
Epoch 200: loss=385.26826344679296, k=[[2.03992477]], b=[0.69125836]
Epoch 300: loss=328.2517944797553, k=[[2.03959199]], b=[0.82006933]
Epoch 400: loss=413.48839302558264, k=[[1.99113144]], b=[0.8829619]
Epoch 500: loss=382.5397537478649, k=[[2.01100017]], b=[0.93458906]
Epoch 600: loss=435.5794218697961, k=[[1.98739557]], b=[1.01688565]
Epoch 700: loss=453.8004157614479, k=[[1.98157274]], b=[1.03336836]
Epoch 800: loss=348.08688809808166, k=[[2.02365658]], b=[1.05184482]
Epoch 900: loss=386.0801130190662, k=[[1.9775763]], b=[1.04739293]
Epoch 1000: loss=353.1195900328757, k=[[1.96046547]], b=[1.04601709]
Epoch 1100: loss=367.66120716505327, k=[[1.9538338]], b=[1.04458963]
Epoch 1200: loss=400.8933636698966, k=[[1.97361014]], b=[1.07146353]
Epoch 1300: loss=399.27191244584264, k=[[1.98046705]], b=[1.06778457]
Epoch 1400: loss=509.3648788431095, k=[[1.98125481]], b=[1.04335596]
Epoch 1500: loss=349.08913021188266, k=[

We got approximate values `k = 1.96` and `b = 1.06`.


## 2. Logistic Regression using TF2 Keras api

- Generate some data;
- Define simple logistic regression (1 dense layer with sigmoid activation);
- Train model

In [4]:
# generate data
def sampler(n, x, y):
    return np.random.normal(size=[n,2]) + [x, y]

def sample_data(n=1000, p0=(-1., -1.), p1=(1., 1.)):
    zeros, ones = np.zeros((n, 1)), np.ones((n, 1))
    labels = np.vstack([zeros, ones])
    
    z_sample = sampler(n, x=p0[0], y=p0[1])
    o_sample = sampler(n, x=p1[0], y=p1[1])
    
    return np.vstack([z_sample, o_sample]), labels

X_train, Y_train = sample_data()
X_test, Y_test = sample_data(100)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(2000, 2) (2000, 1)
(200, 2) (200, 1)


In [5]:
# define model
model = tf.keras.Sequential(
    tf.keras.layers.Dense(1, activation='sigmoid')
)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [8]:
# train model
model.fit(X_train, Y_train, epochs=30, batch_size=16, validation_data=(X_test, Y_test))

Train on 2000 samples, validate on 200 samples
Epoch 1/30
2000/2000 [==============================] - 0s 45us/sample - loss: 0.2009 - accuracy: 0.9185 - val_loss: 0.1874 - val_accuracy: 0.9100
Epoch 2/30
2000/2000 [==============================] - 0s 42us/sample - loss: 0.2009 - accuracy: 0.9185 - val_loss: 0.1874 - val_accuracy: 0.9100
Epoch 3/30
2000/2000 [==============================] - 0s 41us/sample - loss: 0.2009 - accuracy: 0.9190 - val_loss: 0.1874 - val_accuracy: 0.9100
Epoch 4/30
2000/2000 [==============================] - 0s 43us/sample - loss: 0.2009 - accuracy: 0.9185 - val_loss: 0.1874 - val_accuracy: 0.9100
Epoch 5/30
2000/2000 [==============================] - 0s 43us/sample - loss: 0.2009 - accuracy: 0.9185 - val_loss: 0.1874 - val_accuracy: 0.9100
Epoch 6/30
2000/2000 [==============================] - 0s 42us/sample - loss: 0.2009 - accuracy: 0.9180 - val_loss: 0.1874 - val_accuracy: 0.9100
Epoch 7/30
2000/2000 [==============================] - 0s 42us/sample 

We got `train accuracy = 0.9185` and `validation accuracy = 0.9100`